In [1]:
import districting_methods
import geopandas as gpd
import networkx as nx
import pickle as pkl
import concurrent
import os

* `poland` - shapefile of Poland
* `graph` - graph of Poland's voting districts (adjency list based on Voronoi algorithm - details can be found in `shapefiles/wybory2019_voronoi`)

In [2]:
poland = gpd.read_file("shapefiles/polska/polska.shp", encoding="utf-8")
graph = nx.read_gexf("pickle_files/wybory2019sejm_graph_without_parties.gexf")

#### The following cell is optional
If you want to carry out the coarsening of the graph (reduce the number of nodes by merging some adjacent nodes), run the following code:

In [3]:
graph = districting_methods.graph_coarsening(graph, decrease_percent=0.5, num_of_iterations=1)[0]

Coarsening iteration 1/1
Coarsening progress:  0.3105590062111801 %
Coarsening progress:  0.6556245686680469 %
Coarsening progress:  5.391649413388544 %
Coarsening progress:  5.805728088336784 %
Coarsening progress:  10.412353347135955 %
Coarsening progress:  10.84368530020704 %
Coarsening progress:  15.053485162180813 %
Coarsening progress:  15.484817115251898 %
Coarsening progress:  15.916149068322982 %
Coarsening progress:  20.18633540372671 %
Coarsening progress:  20.617667356797792 %
Coarsening progress:  25.293305728088338 %
Coarsening progress:  25.724637681159418 %
Coarsening progress:  30.400276052449964 %
Coarsening progress:  30.814354727398207 %
Coarsening progress:  35.06728778467909 %
Coarsening progress:  35.49861973775017 %
Coarsening progress:  35.929951690821255 %
Coarsening progress:  40.19151138716356 %
Coarsening progress:  40.61421670117322 %
Coarsening progress:  45.30710835058661 %
Coarsening progress:  45.7384403036577 %
Coarsening progress:  50.414078674948236

Preparing geopandas frame (no matter if you've coarsend or not)

In [4]:
gdf = gpd.read_file("shapefiles/wybory2019_voronoi/wybory2019sejm_voronoi.shx", encoding="utf-8")
gdf = gdf[['teryt', 'obwod', 'geometry']]
gdf['id'] = gdf['teryt'] + '_' + gdf['obwod'].astype(str)

simple_id_to_graph_node_map = {}
for graph_node in graph.nodes:
    simple_ids = graph_node.split('+')
    for simple_id in simple_ids:
        simple_id_to_graph_node_map[simple_id] = graph_node

gdf['graph_node_id'] = gdf['id'].map(simple_id_to_graph_node_map)
unmapped_rows = gdf[gdf['graph_node_id'].isnull()]
if not unmapped_rows.empty:
    print(f"\nUWAGA: Nie znaleziono mapowania dla {len(unmapped_rows)} wierszy. ID, których nie było w grafie:")
    print(unmapped_rows['id'].unique())
gdf_dissolved = gdf.dissolve(by='graph_node_id', as_index=False)

In [ ]:
N_OF_THREADS = 5 # number of threads to use in parallel computations
n_districts = 100 # number of districts to create
hot_steps = 500 # number of hot steps in simulated annealing
annealing_steps = 300 # number of annealing steps in simulated annealing
cold_steps = 1000 # number of cold steps in simulated annealing

In [6]:
def run_redist_flip_alg(beta_eq_pop_target, beta_compactness_target, initial_seeding_attempts = 20, initial_districting = None, i=0):
        print(f"Running redist_flip_alg with thread {i + 1}/{N_OF_THREADS}")
        result, data = districting_methods.redist_flip_alg(
            graph, gdf_dissolved, poland, number_of_districts = n_districts,
            hot_steps = hot_steps, annealing_steps = annealing_steps, cold_steps = cold_steps,
            lambda_prob=0.1, beta_eq_pop_target=beta_eq_pop_target, beta_compactness_target=beta_compactness_target, initial_seeding_attempts = initial_seeding_attempts, initial_districts = initial_districting
        )
        print(f"Finished redist_flip_alg with thread {i}/{N_OF_THREADS}")
        if not os.path.exists("results"):
            os.makedirs("results")
        if os.path.exists(f"results/redist_flip_alg_result_{beta_eq_pop_target}_{beta_compactness_target}.pkl") or os.path.exists(f"results/redist_flip_alg_data_{beta_eq_pop_target}_{beta_compactness_target}.pkl"):
            index = 1
            while os.path.exists(f"results/redist_flip_alg_result_{beta_eq_pop_target}_{beta_compactness_target}_{index}.pkl") or os.path.exists(f"results/redist_flip_alg_data_{beta_eq_pop_target}_{beta_compactness_target}_{index}.pkl"):
                index += 1
            pkl.dump(result, open(f"results/redist_flip_alg_result_{beta_eq_pop_target}_{beta_compactness_target}_{index}.pkl", "wb"))
            pkl.dump(data, open(f"results/redist_flip_alg_data_{beta_eq_pop_target}_{beta_compactness_target}_{index}.pkl", "wb"))
        else:
            pkl.dump(result, open(f"results/redist_flip_alg_result_{beta_eq_pop_target}_{beta_compactness_target}.pkl", "wb"))
            pkl.dump(data, open(f"results/redist_flip_alg_data_{beta_eq_pop_target}_{beta_compactness_target}.pkl", "wb"))

Here you can set parameters for the algorithm. Each tuple in a list is a separate job that will be run in parallel. Pair is in format: 

(`beta_eq_pop_target, beta_compactness_target, initial_seeding_attempts, initial_districting`)

* higher `beta_eq_pop_target` means that the algorithm will prioritize equal population more
* higher `beta_compactness_target` means that the algorithm will prioritize compactness more
* `initial_seeding_attempts` - number of attempts to create initial districting (algorithm creates initial districting randomly, and then picks the best one - 20 by default)
* `initial_districting` - if you want to provide your own initial districting (fe. from previous run), provide it here. If provided, `initial_seeding_attempts` is ignored. By default, results are saved in `results` folder with filenames indicating the parameters used. If you want to use `initial_seeding_attempts`, set `initial_districting` to `None`.

In [7]:
## Example usage with initial_districting:
    # initial_districting = pkl.load(open("results/redist_flip_alg_result_3100_3020.pkl", "rb"))
    # job_params = [(3100, 3020, 0, initial_districting)]
## Additionaly - I would suggest changing values of hot_steps, annealing_steps and cold_steps accordingly
## If you are using initial_districting, then I would suggest lowering number of hot_steps and annealing_steps
## That is because if algorithm is in hot phase or annealing phase, it has higher chance of doing random changes, even when they aren't benefitial
## and when you're using initial_districting, I would assume you want to make it better, not change it randomly
## So, for example, for seeding attempts use:
    # hot_steps, annealing_steps, cold_steps = 500, 300, 1000
## And with initial_districting change that to fe:
    # hot_steps, annealing_steps, cold_steps = 50, 30, 10000

job_params = [(3100, 3020, 10, None), (3000, 2700, 10, None)]

In [8]:
with concurrent.futures.ThreadPoolExecutor(max_workers=N_OF_THREADS) as executor:
    futures = [executor.submit(run_redist_flip_alg, beta_eq, beta_comp, init_seed, init_dist, i) for i, (beta_eq, beta_comp, init_seed, init_dist) in enumerate(job_params)]
    
    for future in concurrent.futures.as_completed(futures):
        try:
            result = future.result()
        except Exception as exc:
            print(f'An exception has been raised: {exc}')

Running redist_flip_alg with thread 1/5Running redist_flip_alg with thread 2/5

Finding the best initial seeding out of 10 attempts...
Finding the best initial seeding out of 10 attempts...
Error in seeding algorithm: Unexpected error in seeding algorithm - node is not connected to any of the covered nodes. Retrying...
Starting redistricting algorithm with 100 districts, hot steps: 10, annealing steps: 10, cold steps: 100, lambda probability: 0.1, beta_eq_pop_target: 3000, beta_compactness_target: 2700
Failed to divide the largest district into two connected subgraphs after 20 attempts.
Starting redistricting algorithm with 100 districts, hot steps: 10, annealing steps: 10, cold steps: 100, lambda probability: 0.1, beta_eq_pop_target: 3100, beta_compactness_target: 3020
Step 100/120
Avg Derivation: 17.719658552696586, Compactness: 73.60737266846715
Accepted: 45, Rejected: 55
pop_target: 3000.0, comp_target: 2700.0
Final Avg Derivation: 17.64004509550494, Final Compactness: 73.545399718